In [1]:
import pandas as pd

E = [[0,0,1,0], [0,0,1,0]]
print(E[1][2])
E_dict = pd.DataFrame(E).T.to_dict()
print(E_dict)
print(E_dict[1][2])
H = [1,0,0,1,1,1,1,1]
print(pd.Series(H, index=range(1, 9)).to_dict())

1
{0: {0: 0, 1: 0, 2: 1, 3: 0}, 1: {0: 0, 1: 0, 2: 1, 3: 0}}
1
{1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}
